In [ ]:
from __future__ import division
from Fix_data import fix_data
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import matplotlib.table as tb
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras import optimizers

#this class is used for making predictions using the different models
class making_preds:

    def optim_model(model_type, X_train, y_train, X_valid, y_valid, num, shift_ord):
        
        if model_type != 'LSTM':
         #normalize data to stabilize SVR
            rbX = RobustScaler()
            X_train1 = pd.DataFrame(rbX.fit_transform(X_train.iloc[num:,1:]))
            X_train1 = X_train1.set_index(X_train[num:].index)
            X_train = pd.concat([pd.DataFrame(X_train.iloc[num:,0]), X_train1], axis=1)
            rbY = RobustScaler()
            y_train = rbY.fit_transform(y_train[num:])
            
            #normalize validation data 
            X_valid1 = pd.DataFrame(rbX.transform(X_valid.iloc[shift_ord:,1:]))
            X_valid1 = X_valid1.set_index(X_valid[shift_ord:].index)
            X_valid = pd.concat([pd.DataFrame(X_valid.iloc[shift_ord:,0]), X_valid1], axis=1)
            
        if model_type == 'BENCHMARK':
            model = sm.OLS(y_train[num:], X_train[num:])
            model = model.fit()
            
        elif model_type == 'MEAN':
            from sklearn.neighbors import KNeighborsRegressor
            k = len(y_train[num:])
            
            model = KNeighborsRegressor(n_neighbors=k)
            model.fit(X_train[num:],y_train[num:])

        elif model_type == 'RIDGE':
            #define the ridge reg model
            #set up different alpha values to try
            alpha_ = (.0001, .001, .01, .1, 1, 10, 100, 1000)
            rmse_valid = np.zeros([len(alpha_)])

            #create loop to try different alphas and store validation error to pick best one
            i = 0
            for a in range(len(alpha_)):
                reg = Ridge(alpha=alpha_[a])
                reg.fit(X_train[num:],y_train[num:])
                ridge_preds = reg.predict(X_train[num:])

                #####################validation########################
                valid_ridge_preds = pd.DataFrame(reg.predict(X_valid[shift_ord:]))
                valid_ridge_preds = valid_ridge_preds.set_index(y_valid[shift_ord:].index)
                valid_ridge_preds = rbY.inverse_transform(valid_ridge_preds)
                rmse_valid[i] = np.sqrt(mean_squared_error(y_valid[shift_ord:], valid_ridge_preds)) #/ np.abs(np.mean(y_valid))
                i += 1

            ###########################################################################
            #now use the model with the best validation RMSE###########################
            plt.plot(np.arange(len(alpha_)),rmse_valid)
            best_a=  np.argmin(rmse_valid)
            best_alpha = alpha_[best_a]
            model = Ridge(alpha=best_alpha)
            model.fit(X_train, y_train)

        elif model_type == 'KNN':
            from sklearn.neighbors import KNeighborsRegressor
            k = np.arange(60)
            rmse_valid = np.zeros([len(k)])
            #create loop to try differnet alphas and store validation error to pick best one
            i = 0
            for a in range(len(k)):
                mod = KNeighborsRegressor(n_neighbors=a+1)
                mod.fit(X_train[num:],y_train[num:])
                knn_preds = mod.predict(X_train[num:])

                #####################validation########################
                valid_knn_preds = pd.DataFrame(mod.predict(X_valid[shift_ord:]))
                valid_knn_preds = valid_knn_preds.set_index(y_valid[shift_ord:].index)
                valid_knn_preds = rbY.inverse_transform(valid_knn_preds)
                rmse_valid[i] = np.sqrt(mean_squared_error(y_valid[shift_ord:], valid_knn_preds)) #/ np.abs(np.mean(y_valid))
                i += 1

            ###########################################################################
            #now use the model with the best validation RMSE###########################
            plt.plot(k, rmse_valid)
            best_k =  np.argmin(rmse_valid) + 1
            model = KNeighborsRegressor(n_neighbors=best_k)
            model.fit(X_train, y_train)

        elif model_type == 'RF':
            from sklearn.ensemble import RandomForestRegressor
            feat = (X_train.shape[1] - 1)
            trees = (10,25,50,75,100,150,250,500,1000)
            max_feats = np.arange(1,feat) #(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)
            depth = (2,3,4,5,6)

            rmse_valid = np.zeros([len(trees), len(max_feats), len(depth)])
            for t in range(len(trees)):
                for s in range(len(max_feats)):
                    for d in range(len(depth)):

                        mod = skl.ensemble.RandomForestRegressor(n_estimators=trees[t], max_features=max_feats[s], max_depth=depth[d], random_state=0)
                        mod.fit(X_train, np.ravel(y_train))

                        #####################validation########################
                        valid_rf_preds = pd.DataFrame(mod.predict(X_valid[shift_ord:]))
                        valid_rf_preds = valid_rf_preds.set_index(y_valid[shift_ord:].index)
                        valid_rf_preds = rbY.inverse_transform(valid_rf_preds)
                        rmse_valid[t,s,d] = np.sqrt(mean_squared_error(y_valid[shift_ord:], valid_rf_preds)) 

            ###########################################################################
            #now use the model with the best validation RMSE###########################
            flat_index = np.argmin(rmse_valid)
            best_combo =  np.unravel_index(flat_index, rmse_valid.shape)
            best_t = best_combo[0]
            best_s = best_combo[1]
            best_d = best_combo[2]
            model = skl.ensemble.RandomForestRegressor(n_estimators=trees[best_t], max_features=max_feats[best_s], max_depth=depth[best_d], random_state=0)
            model.fit(X_train, np.ravel(y_train))


        elif model_type == 'SVR':
            from sklearn.svm import SVR
#             #normalize data to stabilize SVR
#             rbX = RobustScaler()
#             X_train1 = pd.DataFrame(rbX.fit_transform(X_train.iloc[num:,1:]))
#             X_train1 = X_train1.set_index(X_train[num:].index)
#             X_train = pd.concat([pd.DataFrame(X_train.iloc[num:,0]), X_train1], axis=1)
#             rbY = RobustScaler()
#             y_train = rbY.fit_transform(y_train[num:])
            
#             #normalize validation data 
#             X_valid1 = pd.DataFrame(rbX.transform(X_valid.iloc[shift_ord:,1:]))
#             X_valid1 = X_valid1.set_index(X_valid[shift_ord:].index)
#             X_valid = pd.concat([pd.DataFrame(X_valid.iloc[shift_ord:,0]), X_valid1], axis=1)

            #set parameter values for cross validation
            C_vals = (.0001, .0005, .001, .005,.01, .05, .1, .5, 1, 5, 10, 50, 100,500)
            gam_vals = (.00001, .0001, .001,.01, .1,1)
            rmse_valid = np.zeros([len(C_vals), len(gam_vals)])
           
            #make loop to test for different combinations of hyperparameters using validation set 
            for c in range(len(C_vals)):
                for g in range(len(gam_vals)):
                    model = SVR(kernel='rbf',C=C_vals[c], gamma=gam_vals[g])
                    model.fit(X_train, y_train)

                    valid_svr_preds = pd.DataFrame(model.predict(X_valid))
                    valid_svr_preds = valid_svr_preds.set_index(y_valid[shift_ord:].index)
                    valid_svr_preds = rbY.inverse_transform(valid_svr_preds)
                    rmse_valid[c,g] = np.sqrt(mean_squared_error(y_valid[shift_ord:], valid_svr_preds))
                    
            #now use the model with the best validation RMSE###########################
            flat_index = np.argmin(rmse_valid)
            best_combo =  np.unravel_index(flat_index, rmse_valid.shape)
            best_c = best_combo[0]
            best_g = best_combo[1]
            model = SVR(kernel='rbf',C=C_vals[best_c], gamma=gam_vals[best_g])
            model.fit(X_train, y_train)

        elif model_type == 'LSTM':
            print('hello!!!!!')
            #normalize data to stabilize training 
            rbX = StandardScaler()
            #X_train1 = pd.DataFrame(rbX.fit_transform(X_train.iloc[num:,1:]))
            #X_train1 = X_train1.set_index(X_train[num:].index)
            X_train1 = rbX.fit_transform(X_train.iloc[num:,1:])
            X_train = np.concatenate((X_train.iloc[num:,:1], X_train1), axis=1)
            print(X_train.shape)
            train_size = X_train.shape[0]
            dims = X_train.shape[1]
            X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
            print(X_train.shape)
            
            rbY = StandardScaler()
            y_train = rbY.fit_transform(y_train[num:])

            #normalize validation data 
            #X_valid1 = pd.DataFrame(rbX.transform(X_valid.iloc[shift_ord:,1:]))
            #X_valid1 = X_valid1.set_index(X_valid[shift_ord:].index)
            X_valid1 = rbX.transform(X_valid.iloc[shift_ord:,1:])
            X_valid = np.concatenate((X_valid.iloc[shift_ord:,:1], X_valid1), axis=1)
            print(X_valid.shape)
            X_valid = np.reshape(X_valid, (X_valid.shape[0],1,X_valid.shape[1]))
            
            #set range of hyperparameters
            node = (4,8,16,32,64)
            learn_rate = (.0001, .001, .01, .1)
            eps = 100

            rmse_valid = np.zeros([len(node), len(learn_rate)])
            for n in range(len(node)): 
                for rate in range(len(learn_rate)): 
                    model = Sequential()
                    model.add(LSTM(node[n], input_shape=(1, dims)))
                    model.add(Dropout(0.5))
                    #model.add(Dense(node[n], activation = 'tanh'))
                    model.add(Dense(1, activation='linear'))

                    rmsp = optimizers.RMSprop(lr=learn_rate[rate])
                    model.compile(loss='mse',
                                  optimizer=rmsp)
                    model.fit(X_train, y_train, epochs=eps, batch_size=train_size)
                    
                    valid_lstm_preds = pd.DataFrame(model.predict(X_valid))
                    valid_lstm_preds = valid_lstm_preds.set_index(y_valid[shift_ord:].index)
                    valid_lstm_preds = rbY.inverse_transform(valid_lstm_preds)
                    rmse_valid[n,rate] = np.sqrt(mean_squared_error(y_valid[shift_ord:], valid_lstm_preds))
                    
                    
            #now use the model with the best validation RMSE###########################
            flat_index = np.argmin(rmse_valid)
            best_combo =  np.unravel_index(flat_index, rmse_valid.shape)
            best_n = best_combo[0]
            best_rate = best_combo[1]
            
            model = Sequential()
            model.add(LSTM(node[best_n], input_shape=(1, dims)))
            model.add(Dropout(0.5))
            model.add(Dense(node[best_n], activation = 'tanh'))
            model.add(Dense(1, activation='linear'))

            rmsp = optimizers.RMSprop(lr=learn_rate[best_rate])
            model.compile(loss='mse',
                          optimizer=rmsp)
            model.fit(X_train, y_train, epochs=eps, batch_size=1)
            print('best nodes =', best_n, '  best rate = ', best_rate)

        print(model)

        return model

    #input number n points to predict ahead using other predictions
    #also input the true values to make predictions
    #use to make preds if you used only AR values
    def pred_ahead_AR(n, X_in, X_train, y_train, model, model_type):
        #get number of points
            
        if model_type != 'LSTM':
            rbX = RobustScaler()
            X_train = rbX.fit_transform(X_train)
            X_in = pd.DataFrame(rbX.transform(X_in))

        if model_type =='LSTM':
            rbX = StandardScaler()
            X_train = rbX.fit_transform(X_train)
            X_in = pd.DataFrame(rbX.transform(X_in))

        X = X_in.iloc[:,1:].values
        t = X.shape[0]
        num_lags = X.shape[1]

        y_hat = np.zeros([t,n])
        y_hat[:,0] = X[:,0]
        new_X = np.zeros(X.shape)
        
        
        
        if model_type == 'MEAN':
            y_hat[:,:] = np.mean(y_train)

        else:
                #loop through number of months to predict into.
            #each additional month shifts the input data so that we have some
            #of the original data and then also the relevant points that we have forecasted already
            for j in range(n):
                if num_lags >= j+1:
                    new_X = np.concatenate((X[:,j:], y_hat[:,:j]),axis=1)
                    new_X = sm.add_constant(new_X)
                    if model_type =='LSTM':
                        new_X = np.reshape(new_X, (new_X.shape[0], 1, new_X.shape[1]))
                    y_hat[:,j] = (model.predict(new_X)).reshape(t,)
                else:
                    new_X = y_hat[:, (j-num_lags):j]
                    new_X = sm.add_constant(new_X)
                    if model_type =='LSTM':
                        new_X = np.reshape(new_X, (new_X.shape[0], 1, new_X.shape[1]))
                    y_hat[:,j] = (model.predict(new_X)).reshape(t,)

            if model_type != 'LSTM': #or model_type =='LSTM':
                rbY = RobustScaler()
                Y = rbY.fit_transform(y_train)
                y_hat = rbY.inverse_transform(y_hat)

            if model_type =='LSTM':
                rbY = StandardScaler()
                Y = rbY.fit_transform(y_train)
                y_hat = rbY.inverse_transform(y_hat)

        return pd.DataFrame(y_hat)

        #use to make preds if you used a combo of mortgage and AR vals
    def pred_ahead_combo(n, num_lags, X_in,X_train,y_train, model,model_type):
            #get number of points
            if model_type != 'LSTM':
                rbX = RobustScaler()
                X_train = rbX.fit_transform(X_train)
                X_in = pd.DataFrame(rbX.transform(X_in))

            if model_type =='LSTM':
                rbX = StandardScaler()
                X_train = rbX.fit_transform(X_train)
                X_in = pd.DataFrame(rbX.transform(X_in))

            X = X_in.iloc[:,1:].values
            t = X.shape[0]

            y_hat = np.zeros([t,n])
            y_hat[:,0] = X[:,0]
            new_X = np.zeros(X.shape)
            
              
            if model_type == 'MEAN':
                y_hat[:,:] = np.mean(y_train)
        
            else:

                #loop through number of months to predict into.
                #each additional month shifts the input data so that we have some
                #of the original data and then also the relevant points that we have forecasted already
                for j in range(n):
                    
                    if j+1 > num_lags:
                        new_X1 = y_hat[:,j-num_lags:j]
                        new_X = np.concatenate((new_X1, X[:,num_lags:]), axis=1)
                        new_X = sm.add_constant(new_X)
                        if model_type =='LSTM':
                            new_X = np.reshape(new_X, (new_X.shape[0], 1, new_X.shape[1]))
                        y_hat[:,j] = (model.predict(new_X)).reshape(t,)
                    else:
                        new_X1 = np.concatenate((X[:,j:num_lags], y_hat[:,:j]),axis=1)
                        new_X = np.concatenate((new_X1, X[:,num_lags:]), axis=1)
                        new_X = sm.add_constant(new_X)
                        if model_type =='LSTM':
                            new_X = np.reshape(new_X, (new_X.shape[0], 1, new_X.shape[1]))
                        y_hat[:,j] = (model.predict(new_X)).reshape(t,)


                if model_type != 'LSTM':
                    rbY = RobustScaler()
                    Y = rbY.fit_transform(y_train)
                    y_hat = rbY.inverse_transform(y_hat)

                if model_type =='LSTM':
                    rbY = StandardScaler()
                    Y = rbY.fit_transform(y_train)
                    y_hat = rbY.inverse_transform(y_hat)

            return pd.DataFrame(y_hat)
        
   

    def predict_things(model_type, X_train, X_valid, X_test,y_train, y_valid, y_test, y_hat, y_hat_valid, y_hat_test, num, model, month_forecast, shift_ord):

            train_size = len(X_train)
            valid_size = len(X_valid)
            filepath = '/Users/abbysuckow/Desktop/' + model_type +'/' + model_type

            #if we built the model based on shifted data
            if shift_ord > 0:
                preds = pd.DataFrame(model.predict(X_train[num:]))
                preds = preds.set_index(y_train[num:].index)

                
                rmse_train = np.sqrt(mean_squared_error(y_train[num:], preds))  
                MAE_train = mean_absolute_error(y_train[num:], preds)

                #plot the training data
                plt.plot(y_train, color = 'blue')
                plt.plot(preds, color='red')
                plt.title('Shifting Data - Train Predictions ' + str(month_forecast) + ' months ahead')
                plt.show()

                #validation set
                valid_preds = pd.DataFrame(model.predict(X_valid))
                valid_preds = valid_preds.set_index(y_valid.index)

                rmse_valid = np.sqrt(mean_squared_error(y_valid[month_forecast:], valid_preds[month_forecast:])) #/ np.abs(np.mean(y_valid[month_forecast:]))
                MAE_valid = mean_absolute_error(y_valid[month_forecast:], valid_preds[month_forecast:])

                plt.plot(y_valid, color = 'blue')
                plt.plot(valid_preds, color='red')
                plt.title('Shifting Data - Validation Predictions' + str(month_forecast) + '  months ahead')
                plt.show()

                #test set
                test_preds = pd.DataFrame(model.predict(X_test))
                test_preds = test_preds.set_index(y_test.index)

                rmse_test = np.sqrt(mean_squared_error(y_test[month_forecast:], test_preds[month_forecast:])) #/ np.abs(np.mean(y_test[month_forecast:]))
                MAE_test = mean_absolute_error(y_test[month_forecast:], test_preds[month_forecast:])

                plt.plot(y_test, color = 'blue')
                plt.plot(test_preds, color='red')
                plt.title('Shifting Data - Test Predictions' + str(month_forecast) + ' months ahead')
                plt.show()


                #make a table of the error measures
                row_labels = ('RMSE', 'MAE')
                col_labels = ('Training', 'Validation', 'Test')
                error_measures = ([rmse_train, rmse_valid, rmse_test], [MAE_train, MAE_valid, MAE_test])
                fig, ax = plt.subplots()
                fig.set_size_inches(8.5, 2.5)

                # Hide axes
                ax.xaxis.set_visible(False)
                ax.yaxis.set_visible(False)
                plt.title(str( month_forecast) + ' months ahead error measures - shifting data')
                ax.table(cellText=error_measures,rowLabels=row_labels,colLabels=col_labels,loc='center')
                plt.show()

                use = preds
                use_valid = valid_preds
                use_test = test_preds

            #if we want to make predictions on predictions:
            else:

                #create loop to store MSE for different prediction intervals
                y_hat_mse = np.zeros([month_forecast,1])
                for f in range(month_forecast):
                    y_hat_mse[f,0] = np.sqrt(mean_squared_error(y_train.iloc[num+f+1:], y_hat.iloc[:][f].shift(f+1)[f+1:])) #/ np.abs(np.mean(y.iloc[num+f+1:train_size]))

                pd.DataFrame(y_hat_mse).plot()
                plt.title('Training RMSE for different month prediction intervals')
                plt.xlabel('Number of Months into the Future')
                plt.ylabel('RMSE')
                plt.savefig(filepath+'_RMSE_' +str(month_forecast))
                plt.show()


                use = pd.DataFrame(y_hat[month_forecast-1])
                use = use.set_index(y_train[num:].index + pd.DateOffset(months=month_forecast))
                
                
                rmse_train= (np.sqrt(mean_squared_error(y_train.iloc[num+month_forecast:], use[:len(use)-month_forecast])))  
                nrmse_train = "{0:.5f}".format(rmse_train / np.std(y_train.iloc[num+month_forecast:])[0])
                rmse_train = "{0:.5f}".format(rmse_train)
                MAE_train = "{0:.5f}".format(mean_absolute_error(y_train.iloc[num+month_forecast:], use[:len(use)-month_forecast]))

                #plot
                plt.plot(y_train[num+month_forecast:], color = 'blue')
                plt.plot(use, color='red')
                plt.title('Training Predictions ' + str(month_forecast) + ' Month(s) Ahead')
                plt.savefig(filepath+'_train_' +str(month_forecast))
                plt.show()

                #validation set
                use_valid = pd.DataFrame(y_hat_valid[month_forecast-1])
                use_valid = use_valid.set_index(y_valid.index + pd.DateOffset(months=month_forecast))

                 
                rmse_valid = (np.sqrt(mean_squared_error(y_valid[month_forecast:], use_valid[:len(use_valid)-month_forecast])))
                nrmse_valid = "{0:.5f}".format(rmse_valid / np.std(y_valid.iloc[month_forecast:])[0])
                rmse_valid = "{0:.5f}".format(rmse_valid)
                MAE_valid = "{0:.5f}".format(mean_absolute_error(y_valid[month_forecast:], use_valid[:len(use_valid)-month_forecast]))

                plt.plot(y_valid[month_forecast:], color = 'blue')
                plt.plot(use_valid, color='red')
                plt.title('Validation Predictions ' + str(month_forecast) + ' Month(s) Ahead')
                plt.savefig(filepath+'_valid_' +str(month_forecast))
                plt.show()

                #test set
                use_test = pd.DataFrame(y_hat_test[month_forecast-1])
                use_test = use_test.set_index(y_test.index + pd.DateOffset(months=month_forecast))


                rmse_test = (np.sqrt(mean_squared_error(y_test[month_forecast:], use_test[:len(use_test)-month_forecast])))
                nrmse_test = "{0:.5f}".format(rmse_test / np.std(y_test.iloc[month_forecast:])[0])
                rmse_test = "{0:.5f}".format(rmse_test)
                MAE_test = "{0:.5f}".format(mean_absolute_error(y_test[month_forecast:], use_test[:len(use_test)-month_forecast]))

                plt.plot(y_test[month_forecast:], color = 'blue')
                plt.plot(use_test, color='red')
                plt.title('Test Predictions ' + str(month_forecast) + ' Month(s) Ahead')
                plt.savefig(filepath+'_test_'+str(month_forecast))
                plt.show()

                #make a table of the error measures
                row_labels = ('RMSE', 'NRMSE', 'MAE')
                col_labels = ('Training', 'Validation', 'Test')
                error_measures = ([rmse_train, rmse_valid, rmse_test],[nrmse_train, nrmse_valid, nrmse_test], [MAE_train, MAE_valid, MAE_test])
                fig, ax = plt.subplots()
                fig.set_size_inches(5.5, 2.5)

                # Hide axes
                ax.axis('off')
                ax.axis('tight')
                ax.xaxis.set_visible(False)
                ax.yaxis.set_visible(False)
                plt.title(str( month_forecast) + ' Month(s) Ahead Error Measures', fontsize=10)
                ax.table(cellText=error_measures,rowLabels=row_labels,colLabels=col_labels,  loc='center',bbox=[0, 0, 1, 1])

                plt.savefig(filepath+'_table_'+str(month_forecast), bbox_inches='tight')
                plt.show()


            return  rmse_train, rmse_valid, rmse_test, use, use_valid, use_test

        #this function adjusts the predictions to the original values and creates
        #plots and error measures based on the original scale of the data.
    def adjusted_preds(model_type, y_hat, y_hat_valid, y_hat_test, CS_dat, perc_adj_CS_dat, dif_order, adj_CS_dat, new_PCE, train_size, valid_size, month_forecast, num):

        filepath = '/Users/abbysuckow/Desktop/' + model_type +'/' + model_type

        unadj_y_hat = fix_data.switch_back(y_hat,perc_adj_CS_dat[num-dif_order+month_forecast:], dif_order,month_forecast,adj_CS_dat[num:], new_PCE[month_forecast+num:train_size+month_forecast])
        unadj_y_hat = pd.DataFrame(unadj_y_hat).set_index(new_PCE[month_forecast+num:train_size+month_forecast].index)
        un = pd.DataFrame(fix_data.switch_back(y_hat,perc_adj_CS_dat[num-dif_order+month_forecast:], dif_order,month_forecast,adj_CS_dat[num:], new_PCE[month_forecast+num:train_size+month_forecast]))
        un = un.set_index(y_hat.index)
        rmse_train= np.sqrt(mean_squared_error(CS_dat.iloc[num+month_forecast:train_size+month_forecast], unadj_y_hat))
        MAE_train = mean_absolute_error(CS_dat.iloc[num+month_forecast:train_size+month_forecast], unadj_y_hat)

        unadj_y_hat_v = fix_data.switch_back(y_hat_valid,perc_adj_CS_dat[train_size-dif_order+month_forecast:], dif_order,month_forecast,adj_CS_dat[train_size:], new_PCE[month_forecast+train_size:train_size+month_forecast+valid_size])
        unadj_y_hat_v = pd.DataFrame(unadj_y_hat_v).set_index(new_PCE[month_forecast+train_size:train_size+month_forecast+valid_size].index)
        rmse_valid = np.sqrt(mean_squared_error(CS_dat.iloc[train_size+month_forecast:train_size+valid_size+month_forecast], unadj_y_hat_v))
        MAE_valid = mean_absolute_error(CS_dat.iloc[train_size+month_forecast:train_size+valid_size+month_forecast], unadj_y_hat_v)

        unadj_y_hat_t = fix_data.switch_back(y_hat_test[:len(new_PCE[month_forecast+train_size+valid_size:])],perc_adj_CS_dat[train_size+valid_size-dif_order+month_forecast:], dif_order,month_forecast,adj_CS_dat[train_size+valid_size:], new_PCE[month_forecast+train_size+valid_size:])
        unadj_y_hat_t = pd.DataFrame(unadj_y_hat_t).set_index(new_PCE[month_forecast+train_size+valid_size:].index)
        rmse_test = np.sqrt(mean_squared_error(CS_dat.iloc[train_size+valid_size+month_forecast:len(CS_dat)-1],unadj_y_hat_t[:len(new_PCE[month_forecast+train_size+valid_size:])]))
        MAE_test = mean_absolute_error(CS_dat.iloc[train_size+valid_size+month_forecast:len(CS_dat)-1],unadj_y_hat_t[:len(new_PCE[month_forecast+train_size+valid_size:])])


        #plots the predictions
        #plt.plot(unadj_y_hat)
        plt.plot(un)
        plt.plot(unadj_y_hat_v)
        plt.plot(unadj_y_hat_t)
        plt.plot(CS_dat[num+month_forecast:])

        plt.ylim(50,300)
        plt.savefig(filepath+'_unadj_'+str(month_forecast))
        plt.show()

        #error tables
        row_labels = ('RMSE', 'MAE')
        col_labels = ('Training', 'Validation', 'Test')
        error_measures = ([rmse_train, rmse_valid, rmse_test], [MAE_train, MAE_valid, MAE_test])
        fig, ax = plt.subplots()
        fig.set_size_inches(8.5, 2.5)

        # Hide axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        plt.title(str( month_forecast) + ' month(s) ahead error measures')
        ax.table(cellText=error_measures,rowLabels=row_labels,colLabels=col_labels, bbox=[0, -0.3, 1, 0.275])
        plt.show()

        return unadj_y_hat, unadj_y_hat_v, unadj_y_hat_t


    def slope_analysis(y_hat, y_tru,model_type,month_forecast):
        filepath = '/Users/abbysuckow/Desktop/' + model_type +'/' + model_type
        yhmax = np.amax(y_hat)
        yhmin = np.min(y_hat)

        ytmax = np.amax(y_tru)
        ytmin = np.min(y_tru)

        if yhmax > ytmax[0]:
            ymax = yhmax +.1*yhmax
        else:
            ymax = ytmax[0] + .1*(ytmax[0])
        if yhmin < ytmin[0]:
            ymin = yhmin - .1*yhmin
        else:
            ymin = ytmin[0] - .1*(ytmin[0])

        if ymax > 1.5:
            step = 1
        else:
            step = .1*ymax


        plt.scatter(y_tru, y_hat)
        plt.plot(np.arange(ymin,ymax,step=step), np.arange(ymin,ymax, step=step))
        plt.xlabel('Truth')
        plt.ylabel('Predictions')
        plt.title('Predictions versus Truth')
        plt.xlim(ymin, ymax)
        plt.ylim(ymin, ymax)
        plt.savefig(filepath+'_slope_'+str(month_forecast))
        plt.show()

        y_hat = sm.add_constant(y_hat)
        model = sm.OLS(y_tru, y_hat)
        model = model.fit()
        print(model.summary())

        plt.rc('figure', figsize=(12, 7))
        plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(filepath+'_ols_'+str(month_forecast))

        return
